In [3]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [4]:
from Chempy import localpath
from Chempy.parameter import ModelParameters
a = ModelParameters()


type = 'delay' # or rapid

if type == 'delay':
    name=localpath+'input/yields/NuGrid/H NuGrid yields delay_total.txt'
    remnants = {}
    remnants[0.02] = [1.61,1.61,2.73,5.71]
    remnants[0.01] = [1.61,1.61,2.77,6.05]
    remnants[0.006] = [1.62,1.62,2.79,6.18]
    remnants[0.001] = [1.62,1.62,2.81,6.35]
    remnants[0.0001] = [1.62,1.62,2.82,6.38]
elif type == 'rapid':
    name = localpath+'input/yields/NuGrid/H NuGrid yields rapid total.txt'
    remants = {}
    remnants[0.02] = [1.44,1.44,2.70,12.81] # Define remnants from metallicities
    remnants[0.01] = [1.44,1.44,1.83,9.84]
    remnants[0.006] = [1.44, 1.44, 1.77, 7.84]
    remnants[0.001] = [1.44,1.44,1.76,5.88]
    remnants[0.0001] = [1.44,1.44,1.76,5.61]
else:
    raise ValueError('Wrong type: must be delay or rapid')
    

# Create list of masses and metallicites:
masses = [12.0,15.0,20.0,25.0]
metallicities = [0.02,0.01,0.006,0.001,0.0001]


#elements_list = ['&'+el for el in list_of_elements]

cuts={}
for z in metallicities:
    cuts[z] = [] 
    for mass in masses:
        file=open(name,"r")
        for line_no,line in enumerate(file):
            if str(mass) in line and str(z) in line:
                cuts[z].append(line_no)
                
line_end = line_no
#index = 0

# Create list of elements
data = np.genfromtxt(name,skip_header=int(cuts[0.02][0])+4,
                             skip_footer=line_end-int(cuts[0.02][0])-83,
                    dtype=['<U8','<U15','<U15','<U15'])
list_of_elements = [str(line[0][1:]) for line in data]

table={}

for z in metallicities:
    
    yield_subtable={}
    yield_subtable['Mass'] = masses
    yield_subtable['mass_in_remnants'] = []
    for el in list_of_elements:
            yield_subtable[el] = []
             
    for m_index,mass in enumerate(masses): 
        
        #subtable_net_fraction={}
        unprocessed_mass = mass-remnants[z][m_index] # in Msun
        
        yield_subtable['mass_in_remnants'].append(remnants[z][m_index]/mass)
        data = np.genfromtxt(name,skip_header=int(cuts[z][m_index])+4,
                             skip_footer=line_end-int(cuts[z][m_index])-83,
                    dtype=['<U8','<U15','<U15','<U15'])
        for line in data:
            el_name = str(line[0][1:]) # Name of element - only do this for relevant elements
            el_yield = float(line[1][1:]) # Yield in Msun
            el_init = float(line[2][1:]) # Initial mass fraction 
            
            el_net=el_yield-el_init*unprocessed_mass # Net yield in Msun
            
            yield_subtable[el_name].append(el_net/unprocessed_mass) # Net mass fraction
            # Net fractions+ remnant fraction do not sum to 1 - this is due to absence of elements e.g. Be
        #index+=1
        #for el in a.elements_to_trace:
        #    fraction_list = [subtable_net_fraction[el]]
    summed_yields = np.zeros(len(masses))
    for el in list_of_elements:
        yield_subtable[el] = np.asarray(yield_subtable[el])
        summed_yields+=yield_subtable[el]

    yield_subtable['mass_in_remnants'] = np.asarray(yield_subtable['mass_in_remnants'])
    yield_subtable['unprocessed_mass_in_winds'] = 1.0-yield_subtable['mass_in_remnants']-summed_yields
    
    
    #table[z] = yield_subtable
    
    all_keys = ['Mass','mass_in_remnants','unprocessed_mass_in_winds']+list_of_elements
    
    list_of_arrays = [yield_subtable[key] for key in all_keys]
    
    restructure_subtable = np.core.records.fromarrays(list_of_arrays,names=all_keys)
    
    table[z] = restructure_subtable

In [233]:
#TEST
z=metallicities[-1]
m_index = 3
mass = masses[m_index]
print(mass)
data = np.genfromtxt(name,skip_header=int(cuts[z][m_index])+4,
                             skip_footer=line_end-int(cuts[z][m_index])-83,
                    dtype=['<U8','<U15','<U15','<U15'])
for line in data[:1]:
    el_name = str(line[0][1:]) # Name of element - only do this for relevant elements
    el_yield = float(line[1][1:]) # Yield in Msun
    el_init = float(line[2][1:]) # Initial mass fraction 
net=el_yield-el_init*mass
print(net/mass)

25.0
-0.38036000000000003


In [236]:
sum=0
for el in list_of_elements:
    #print(el,table[0.02][el])
    sum+=table[0.02][el]
print(sum+table[0.02]['unprocessed_mass_in_winds']+table[0.02]['mass_in_remnants'])



[ 1.  1.  1.  1.]


In [229]:
print(np.sum(list(test.values()),axis=0)+table[0.0001]['mass_in_remnants']*masses)
print(masses)

[ 11.99991889  14.99972546  20.00041963  24.99943411]
[12.0, 15.0, 20.0, 25.0]


In [128]:
np.sum(list(subtable_yield.values()))+remnant_mass*mass
print(subtable_yield.keys())

dict_keys([])


In [231]:
total=0
for el in list_of_elements:
    total+=table[0.02][el]
    
total+table[0.02]['mass_in_remnants']

array([  2.52820208e-05,  -8.93036250e-06,  -2.66067166e-06,
         1.56915585e-05])

In [249]:
table[0.0001]

rec.array([ ( 12.,  0.135 ,  0.86500781, -0.22259461,  0.13791618,  -6.84013102e-14,  -5.13105973e-13,  0.01179864,   1.97837842e-05,  0.02717058,   2.08965241e-09,  0.00063658,   1.70563474e-06,  0.00157636,   5.92645948e-06,  0.00894736,   1.10044477e-05,  -1.07082852e-12,  0.00524551,   7.31649120e-06,   1.04317300e-03,   6.78578073e-06,   8.36678441e-04,   1.62560219e-08,   6.63037669e-05,   1.43600098e-06,   3.24353976e-04,   3.84764859e-05,   2.35342065e-02,   2.63387142e-04,   3.11359508e-03,   1.15500953e-05,   9.80494077e-06,   2.85457007e-08,   4.16840162e-08,   4.08634104e-11,   1.00233545e-09,   2.97144509e-12,   2.86050096e-11,   1.89768786e-12,   8.77720617e-12,   1.93892100e-12,   3.01443160e-12,   1.75071291e-13,   9.44701349e-14,  -3.22335260e-13,  -1.29121387e-13,  -1.29236994e-13,  -1.65728324e-13,  -1.05909441e-13,  -6.98341040e-14,  -5.46531792e-13,  -8.89152216e-14,  -1.83649326e-12,  -1.84470135e-13,  -1.03053950e-12,  -6.28381503e-14,  -1.07834297e-12,  -1.30186

In [3]:
from Chempy.yields import SN2_feedback
basic_sn2 = SN2_feedback()
getattr(basic_sn2,'chieffi04_net')()
from Chempy.yields import SN2_feedback
basic_sn3 = SN2_feedback()
getattr(basic_sn3,'NuGrid_net')()

In [6]:
basic_sn2.table[0.001]

for key in basic_sn2.table[0.001].dtype.names:
    if key in basic_sn3.table[0.001].dtype.names and key !='B' and key !='Li':
        print(key,basic_sn2.table[0.001][2][key],basic_sn3.table[0.001][2][key])

Mass 20.0 20.0
mass_in_remnants 0.0775 0.1405
unprocessed_mass_in_winds 0.922499993992 0.859479465699
Al 0.000217889547806 8.89382691681e-05
Ar 0.000433646525587 0.00120297421524
As 7.92206614577e-11 1.3486917452e-08
Br 3.25771352163e-11 7.40690040721e-09
C 0.014310724074 0.0128007198371
Ca 0.00039598591216 0.000807378656777
Cl 2.13557769225e-06 9.90865e-06
Co 5.58930253732e-06 2.05367044793e-07
Cr 8.45467535997e-05 0.000342152003607
Cu 2.05213963148e-07 2.1449794299e-07
F 1.23539954714e-09 1.06783571844e-08
Fe 0.00521216354403 0.00236431855148
Ga 2.04663386337e-09 4.19850997673e-08
Ge 7.56396491933e-09 1.31158373473e-07
H -0.215036093954 -0.283622280396
He 0.0811782098707 0.120224374636
K 8.85631049099e-07 7.50308435137e-06
Kr 8.9261658718e-10 5.32669924375e-08
Mg 0.00797620683444 0.00591108503781
Mn 2.41578982502e-05 3.33223280396e-05
Mo 6.21063883402e-09 1.57558772542e-09
N 0.000132531431668 0.000222344293194
Na 0.000155888673143 3.00825388598e-05
Nb 8.88553848148e-13 9.22224909831e

In [258]:
table[0.001][2]['H']

-0.28362228039557885

In [97]:
cuts[0.02]

[930]

In [ ]:
list_of_elements = [H,He,Li,B,C,N,O,F,Ne,Na,Mg,Al,Si,P,Pb,S,Cl,Ar,K,Ca,Sc,Ti,V,Cr.Mn,Fe,Co,Ni,Cu,Zn,Ga,Ge,As,Se,Br,Kr,Rb,Sr,Y,Zr,Nb,Mo,Ru,Rh,Pd,Ag,Cd,In,Sn,Sb,Te,I,Xe,Cs,Ba,La,Ce,Pr,Nd,Sm,Eu,Gd,Tb,Dy,Ho,Er,Tm,Yb,Lu,Hf,Ta,W,Re,Os,Ir,Pt,Au,Hg,Tl,Bi]